In [196]:
import pandas as pd
import os
import csv
from pathlib import Path
import datetime as dt
from uszipcode import SearchEngine

## LOAD LOCATION DATA

In [197]:
zipcodes_coords_df = pd.read_csv("./resources/zipcodes_coordinates.csv")
zipcodes_coords_df

,RegionID,City,State,Zip Code,Latitude,Longitude
0,6181,New York,New York,10001,40.748418,-73.994147
1,17426,Chicago,Illinois,60601,41.885910,-87.623849
2,18959,Las Vegas,Nevada,89101,36.167540,-115.139739
3,38128,Dallas,Texas,75201,32.785918,-96.798987
4,10920,Columbus,Ohio,43085,40.099379,-83.015356
...,...,...,...,...,...,...
942,25643,Longboat Key,Florida,34228,27.392012,-82.641345
943,49592,New Hope,Pennsylvania,18938,40.348777,-74.994791
944,48710,Arnold,California,95223,38.257253,-120.328708
945,12306,Langhorne,Pennsylvania,19047,40.178802,-74.913333


In [198]:
# Create list of viable regionID's
good_id_list = zipcodes_coords_df['RegionID'].tolist()

### Load cleaned (non-transposed) dataset, then remove rows without matching location data.

In [199]:
time_df = pd.read_csv("./resources/Sale_Prices_City/Sale_Prices_City_sample.csv")
time_df

,RegionID,RegionName,StateName,SizeRank,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,...,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03
0,6181,New York,New York,1,462700.0,466200.0,465600.0,469500.0,471300.0,468600.0,...,563200.0,570500.0,572800.0,569900.0,560800.0,571500.0,575100.0,571700.0,568300.0,573600.0
1,17426,Chicago,Illinois,4,188500.0,189400.0,186300.0,177700.0,167900.0,174700.0,...,271500.0,266500.0,264900.0,265000.0,264100.0,264300.0,270000.0,281400.0,302900.0,309200.0
2,18959,Las Vegas,Nevada,8,115600.0,114000.0,110800.0,109600.0,108200.0,107500.0,...,266000.0,268200.0,268400.0,270400.0,272200.0,274400.0,277500.0,278900.0,283000.0,284700.0
3,38128,Dallas,Texas,10,191900.0,184500.0,174500.0,152000.0,162900.0,171400.0,...,300600.0,301100.0,301500.0,310100.0,314600.0,316900.0,307100.0,321500.0,315500.0,321700.0
4,10920,Columbus,Ohio,19,89100.0,86000.0,79800.0,79300.0,78600.0,82300.0,...,152600.0,154800.0,158500.0,157700.0,162900.0,166600.0,172700.0,175300.0,173600.0,177600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,48710,Arnold,California,3704,266300.0,244200.0,217800.0,191200.0,201500.0,208400.0,...,326500.0,317800.0,293100.0,281000.0,279100.0,279400.0,287100.0,305300.0,325600.0,345100.0
1093,12306,Langhorne,Pennsylvania,3714,272300.0,291500.0,305600.0,342300.0,342600.0,329400.0,...,339800.0,371300.0,358600.0,366800.0,362400.0,396600.0,398200.0,404100.0,397100.0,416200.0
1094,33433,Pingree Grove,Illinois,3716,203600.0,175600.0,170200.0,184800.0,198900.0,191800.0,...,206200.0,198200.0,214700.0,211800.0,217500.0,250200.0,270300.0,260200.0,224300.0,216600.0
1095,32133,Idyllwild,California,3718,182900.0,225000.0,230500.0,240100.0,182500.0,183700.0,...,313500.0,309100.0,322400.0,321300.0,342400.0,330500.0,353800.0,355500.0,389300.0,360100.0


In [200]:
# remove rows from time_df that aren't in the list of good locations
for index, row in time_df.iterrows():
    if row['RegionID'] not in good_id_list:
        time_df.drop(index, inplace=True)
        
time_df

,RegionID,RegionName,StateName,SizeRank,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,...,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03
0,6181,New York,New York,1,462700.0,466200.0,465600.0,469500.0,471300.0,468600.0,...,563200.0,570500.0,572800.0,569900.0,560800.0,571500.0,575100.0,571700.0,568300.0,573600.0
1,17426,Chicago,Illinois,4,188500.0,189400.0,186300.0,177700.0,167900.0,174700.0,...,271500.0,266500.0,264900.0,265000.0,264100.0,264300.0,270000.0,281400.0,302900.0,309200.0
2,18959,Las Vegas,Nevada,8,115600.0,114000.0,110800.0,109600.0,108200.0,107500.0,...,266000.0,268200.0,268400.0,270400.0,272200.0,274400.0,277500.0,278900.0,283000.0,284700.0
3,38128,Dallas,Texas,10,191900.0,184500.0,174500.0,152000.0,162900.0,171400.0,...,300600.0,301100.0,301500.0,310100.0,314600.0,316900.0,307100.0,321500.0,315500.0,321700.0
4,10920,Columbus,Ohio,19,89100.0,86000.0,79800.0,79300.0,78600.0,82300.0,...,152600.0,154800.0,158500.0,157700.0,162900.0,166600.0,172700.0,175300.0,173600.0,177600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,25643,Longboat Key,Florida,3690,432300.0,476900.0,440900.0,457500.0,470600.0,489500.0,...,567400.0,554700.0,502000.0,495600.0,535300.0,557900.0,577100.0,557900.0,565600.0,579700.0
1091,49592,New Hope,Pennsylvania,3697,528100.0,482600.0,454800.0,463300.0,475400.0,478600.0,...,545100.0,548300.0,579100.0,556500.0,592700.0,530300.0,579600.0,539400.0,609700.0,608700.0
1092,48710,Arnold,California,3704,266300.0,244200.0,217800.0,191200.0,201500.0,208400.0,...,326500.0,317800.0,293100.0,281000.0,279100.0,279400.0,287100.0,305300.0,325600.0,345100.0
1093,12306,Langhorne,Pennsylvania,3714,272300.0,291500.0,305600.0,342300.0,342600.0,329400.0,...,339800.0,371300.0,358600.0,366800.0,362400.0,396600.0,398200.0,404100.0,397100.0,416200.0


In [201]:
# Merge zipcodes into dataset so we can use them as column headers
time_df = time_df.merge(zipcodes_coords_df, how='inner', on='RegionID')
cols = ['Zip Code', 'RegionID'] + [col for col in time_df.columns if col != 'Zip Code' and col != 'RegionID']
time_df = time_df[cols]
time_df = time_df.dropna()
time_df = time_df.astype({'Zip Code': 'str'})
print(time_df.dtypes)
time_df

Zip Code       object
RegionID        int64
RegionName     object
StateName      object
SizeRank        int64
               ...   
2020-03       float64
City           object
State          object
Latitude      float64
Longitude     float64
Length: 120, dtype: object


,Zip Code,RegionID,RegionName,StateName,SizeRank,2011-01,2011-02,2011-03,2011-04,2011-05,...,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,City,State,Latitude,Longitude
0,10001,6181,New York,New York,1,462700.0,466200.0,465600.0,469500.0,471300.0,...,560800.0,571500.0,575100.0,571700.0,568300.0,573600.0,New York,New York,40.748418,-73.994147
1,60601,17426,Chicago,Illinois,4,188500.0,189400.0,186300.0,177700.0,167900.0,...,264100.0,264300.0,270000.0,281400.0,302900.0,309200.0,Chicago,Illinois,41.885910,-87.623849
2,89101,18959,Las Vegas,Nevada,8,115600.0,114000.0,110800.0,109600.0,108200.0,...,272200.0,274400.0,277500.0,278900.0,283000.0,284700.0,Las Vegas,Nevada,36.167540,-115.139739
3,75201,38128,Dallas,Texas,10,191900.0,184500.0,174500.0,152000.0,162900.0,...,314600.0,316900.0,307100.0,321500.0,315500.0,321700.0,Dallas,Texas,32.785918,-96.798987
4,43085,10920,Columbus,Ohio,19,89100.0,86000.0,79800.0,79300.0,78600.0,...,162900.0,166600.0,172700.0,175300.0,173600.0,177600.0,Columbus,Ohio,40.099379,-83.015356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,34228,25643,Longboat Key,Florida,3690,432300.0,476900.0,440900.0,457500.0,470600.0,...,535300.0,557900.0,577100.0,557900.0,565600.0,579700.0,Longboat Key,Florida,27.392012,-82.641345
943,18938,49592,New Hope,Pennsylvania,3697,528100.0,482600.0,454800.0,463300.0,475400.0,...,592700.0,530300.0,579600.0,539400.0,609700.0,608700.0,New Hope,Pennsylvania,40.348777,-74.994791
944,95223,48710,Arnold,California,3704,266300.0,244200.0,217800.0,191200.0,201500.0,...,279100.0,279400.0,287100.0,305300.0,325600.0,345100.0,Arnold,California,38.257253,-120.328708
945,19047,12306,Langhorne,Pennsylvania,3714,272300.0,291500.0,305600.0,342300.0,342600.0,...,362400.0,396600.0,398200.0,404100.0,397100.0,416200.0,Langhorne,Pennsylvania,40.178802,-74.913333


In [202]:
# recreate transposed dataset
time_df = time_df.drop(columns=["RegionID", "City", "State", "Latitude", "Longitude", "RegionName", "StateName", "SizeRank"])
time_transposed = time_df.transpose()
time_transposed.reset_index(drop = False, inplace = True)
time_transposed = time_transposed.rename(columns=time_transposed.iloc[0])
time_transposed = time_transposed.drop([0, 0])
time_transposed = time_transposed.rename(columns={"Zip Code": 'DATE'})
time_transposed['DATE'] = pd.to_datetime(time_transposed['DATE'], infer_datetime_format=True)
time_transposed

,DATE,10001,60601,89101,75201,43085,40202,32801,80202,20001,...,19468,32550,96738,76262,85641,34228,18938,95223,19047,60081
1,2011-01-01,462700.0,188500.0,115600.0,191900.0,89100.0,125900.0,82000.0,201400.0,401100.0,...,229700.0,284400.0,237800.0,128400.0,204100.0,432300.0,528100.0,266300.0,272300.0,203600.0
2,2011-02-01,466200.0,189400.0,114000.0,184500.0,86000.0,124000.0,82800.0,205300.0,386000.0,...,216700.0,258100.0,216000.0,148400.0,194000.0,476900.0,482600.0,244200.0,291500.0,175600.0
3,2011-03-01,465600.0,186300.0,110800.0,174500.0,79800.0,118300.0,85500.0,198800.0,376200.0,...,199800.0,253800.0,208800.0,150500.0,192400.0,440900.0,454800.0,217800.0,305600.0,170200.0
4,2011-04-01,469500.0,177700.0,109600.0,152000.0,79300.0,117600.0,87800.0,198500.0,362800.0,...,211500.0,225700.0,229800.0,161600.0,185900.0,457500.0,463300.0,191200.0,342300.0,184800.0
5,2011-05-01,471300.0,167900.0,108200.0,162900.0,78600.0,113800.0,88800.0,190900.0,370300.0,...,215300.0,230800.0,260100.0,131500.0,175900.0,470600.0,475400.0,201500.0,342600.0,198900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,2019-11-01,571500.0,264300.0,274400.0,316900.0,166600.0,174600.0,247400.0,445400.0,566400.0,...,264100.0,370700.0,402700.0,343700.0,297200.0,557900.0,530300.0,279400.0,396600.0,250200.0
108,2019-12-01,575100.0,270000.0,277500.0,307100.0,172700.0,176600.0,251100.0,435500.0,577600.0,...,265200.0,412500.0,454400.0,321500.0,292100.0,577100.0,579600.0,287100.0,398200.0,270300.0
109,2020-01-01,571700.0,281400.0,278900.0,321500.0,175300.0,170800.0,254800.0,435100.0,580100.0,...,272300.0,408900.0,499200.0,273700.0,275100.0,557900.0,539400.0,305300.0,404100.0,260200.0
110,2020-02-01,568300.0,302900.0,283000.0,315500.0,173600.0,173100.0,263500.0,436900.0,575700.0,...,248800.0,409000.0,513400.0,314400.0,265400.0,565600.0,609700.0,325600.0,397100.0,224300.0


### LOAD/CLEAN INFLATION DATA

In [203]:
file_path = Path("./resources/CPIAUCNS/CPIAUCNS.csv")
cpi_df = pd.read_csv(file_path)
cpi_df['DATE'] = pd.to_datetime(cpi_df['DATE'], infer_datetime_format=True)
print(cpi_df.dtypes)
cpi_df

DATE        datetime64[ns]
CPIAUCNS           float64
dtype: object


,DATE,CPIAUCNS
0,1913-01-01,9.800
1,1913-02-01,9.800
2,1913-03-01,9.800
3,1913-04-01,9.800
4,1913-05-01,9.700
...,...,...
1304,2021-09-01,274.310
1305,2021-10-01,276.589
1306,2021-11-01,277.948
1307,2021-12-01,278.802


In [204]:
# Merge inflation data into transposed dataset
time_transposed = time_transposed.merge(cpi_df, how='inner', on='DATE')
inf = time_transposed['CPIAUCNS']
time_transposed = time_transposed.drop(columns=['CPIAUCNS'])
time_transposed.insert(loc=1, column='INFLATION', value=inf)
time_transposed = time_transposed.dropna()
time_transposed

,DATE,INFLATION,10001,60601,89101,75201,43085,40202,32801,80202,...,19468,32550,96738,76262,85641,34228,18938,95223,19047,60081
0,2011-01-01,220.223,462700.0,188500.0,115600.0,191900.0,89100.0,125900.0,82000.0,201400.0,...,229700.0,284400.0,237800.0,128400.0,204100.0,432300.0,528100.0,266300.0,272300.0,203600.0
1,2011-02-01,221.309,466200.0,189400.0,114000.0,184500.0,86000.0,124000.0,82800.0,205300.0,...,216700.0,258100.0,216000.0,148400.0,194000.0,476900.0,482600.0,244200.0,291500.0,175600.0
2,2011-03-01,223.467,465600.0,186300.0,110800.0,174500.0,79800.0,118300.0,85500.0,198800.0,...,199800.0,253800.0,208800.0,150500.0,192400.0,440900.0,454800.0,217800.0,305600.0,170200.0
3,2011-04-01,224.906,469500.0,177700.0,109600.0,152000.0,79300.0,117600.0,87800.0,198500.0,...,211500.0,225700.0,229800.0,161600.0,185900.0,457500.0,463300.0,191200.0,342300.0,184800.0
4,2011-05-01,225.964,471300.0,167900.0,108200.0,162900.0,78600.0,113800.0,88800.0,190900.0,...,215300.0,230800.0,260100.0,131500.0,175900.0,470600.0,475400.0,201500.0,342600.0,198900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2019-11-01,257.208,571500.0,264300.0,274400.0,316900.0,166600.0,174600.0,247400.0,445400.0,...,264100.0,370700.0,402700.0,343700.0,297200.0,557900.0,530300.0,279400.0,396600.0,250200.0
107,2019-12-01,256.974,575100.0,270000.0,277500.0,307100.0,172700.0,176600.0,251100.0,435500.0,...,265200.0,412500.0,454400.0,321500.0,292100.0,577100.0,579600.0,287100.0,398200.0,270300.0
108,2020-01-01,257.971,571700.0,281400.0,278900.0,321500.0,175300.0,170800.0,254800.0,435100.0,...,272300.0,408900.0,499200.0,273700.0,275100.0,557900.0,539400.0,305300.0,404100.0,260200.0
109,2020-02-01,258.678,568300.0,302900.0,283000.0,315500.0,173600.0,173100.0,263500.0,436900.0,...,248800.0,409000.0,513400.0,314400.0,265400.0,565600.0,609700.0,325600.0,397100.0,224300.0


### LOAD/CLEAN INTEREST RATE DATA

In [205]:
interest_file = Path("./resources/fedfunds/FEDFUNDS.csv")
interest_df = pd.read_csv(interest_file)
interest_df['DATE'] = pd.to_datetime(interest_df['DATE'], infer_datetime_format=True)
print(interest_df.dtypes)
interest_df

DATE        datetime64[ns]
FEDFUNDS           float64
dtype: object


,DATE,FEDFUNDS
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.07
3,1954-10-01,0.85
4,1954-11-01,0.83
...,...,...
807,2021-10-01,0.08
808,2021-11-01,0.08
809,2021-12-01,0.08
810,2022-01-01,0.08


In [206]:
# merge interest data into transposed dataset
time_transposed = time_transposed.merge(interest_df, how='inner', on='DATE')
interest = time_transposed['FEDFUNDS']
time_transposed = time_transposed.drop(columns=['FEDFUNDS'])
time_transposed.insert(loc=1, column='INTEREST_RATES', value=interest)
time_transposed = time_transposed.dropna()
time_transposed

,DATE,INTEREST_RATES,INFLATION,10001,60601,89101,75201,43085,40202,32801,...,19468,32550,96738,76262,85641,34228,18938,95223,19047,60081
0,2011-01-01,0.17,220.223,462700.0,188500.0,115600.0,191900.0,89100.0,125900.0,82000.0,...,229700.0,284400.0,237800.0,128400.0,204100.0,432300.0,528100.0,266300.0,272300.0,203600.0
1,2011-02-01,0.16,221.309,466200.0,189400.0,114000.0,184500.0,86000.0,124000.0,82800.0,...,216700.0,258100.0,216000.0,148400.0,194000.0,476900.0,482600.0,244200.0,291500.0,175600.0
2,2011-03-01,0.14,223.467,465600.0,186300.0,110800.0,174500.0,79800.0,118300.0,85500.0,...,199800.0,253800.0,208800.0,150500.0,192400.0,440900.0,454800.0,217800.0,305600.0,170200.0
3,2011-04-01,0.10,224.906,469500.0,177700.0,109600.0,152000.0,79300.0,117600.0,87800.0,...,211500.0,225700.0,229800.0,161600.0,185900.0,457500.0,463300.0,191200.0,342300.0,184800.0
4,2011-05-01,0.09,225.964,471300.0,167900.0,108200.0,162900.0,78600.0,113800.0,88800.0,...,215300.0,230800.0,260100.0,131500.0,175900.0,470600.0,475400.0,201500.0,342600.0,198900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2019-11-01,1.55,257.208,571500.0,264300.0,274400.0,316900.0,166600.0,174600.0,247400.0,...,264100.0,370700.0,402700.0,343700.0,297200.0,557900.0,530300.0,279400.0,396600.0,250200.0
107,2019-12-01,1.55,256.974,575100.0,270000.0,277500.0,307100.0,172700.0,176600.0,251100.0,...,265200.0,412500.0,454400.0,321500.0,292100.0,577100.0,579600.0,287100.0,398200.0,270300.0
108,2020-01-01,1.55,257.971,571700.0,281400.0,278900.0,321500.0,175300.0,170800.0,254800.0,...,272300.0,408900.0,499200.0,273700.0,275100.0,557900.0,539400.0,305300.0,404100.0,260200.0
109,2020-02-01,1.58,258.678,568300.0,302900.0,283000.0,315500.0,173600.0,173100.0,263500.0,...,248800.0,409000.0,513400.0,314400.0,265400.0,565600.0,609700.0,325600.0,397100.0,224300.0


### LOAD/CLEAN MORTGAGE RATE DATA

In [207]:
mortgage_file = Path("./resources/mortgage_data_trimmed/Mortgage_data_trimmed.csv")
mortgage_df = pd.read_csv(mortgage_file)
mortgage_df = mortgage_df.rename(columns={'Month': 'DATE', 'Average\r\nContract\r\nRate': 'AVG_MORTGAGE_RATE'})
mortgage_df['DATE'] = pd.to_datetime(mortgage_df['DATE'], infer_datetime_format=True)
print(mortgage_df.dtypes)
mortgage_df

DATE                 datetime64[ns]
AVG_MORTGAGE_RATE           float64
dtype: object


,DATE,AVG_MORTGAGE_RATE
0,NaT,NaN
1,NaT,NaN
2,1998-01-01,7.3
3,1998-02-01,7.2
4,1998-03-01,7.2
...,...,...
276,2020-11-01,3.0
277,2020-12-01,3.0
278,2021-01-01,3.0
279,2021-02-01,3.0


In [208]:
# merge mortgage rate data into transposed dataset
time_transposed = time_transposed.merge(mortgage_df, how='inner', on='DATE')
mort = time_transposed['AVG_MORTGAGE_RATE']
time_transposed = time_transposed.drop(columns=['AVG_MORTGAGE_RATE'])
time_transposed.insert(loc=1, column='AVG_MORTGAGE_RATE', value=mort)
time_transposed = time_transposed.dropna()
time_transposed

,DATE,AVG_MORTGAGE_RATE,INTEREST_RATES,INFLATION,10001,60601,89101,75201,43085,40202,...,19468,32550,96738,76262,85641,34228,18938,95223,19047,60081
0,2011-01-01,4.8,0.17,220.223,462700.0,188500.0,115600.0,191900.0,89100.0,125900.0,...,229700.0,284400.0,237800.0,128400.0,204100.0,432300.0,528100.0,266300.0,272300.0,203600.0
1,2011-02-01,4.9,0.16,221.309,466200.0,189400.0,114000.0,184500.0,86000.0,124000.0,...,216700.0,258100.0,216000.0,148400.0,194000.0,476900.0,482600.0,244200.0,291500.0,175600.0
2,2011-03-01,5.0,0.14,223.467,465600.0,186300.0,110800.0,174500.0,79800.0,118300.0,...,199800.0,253800.0,208800.0,150500.0,192400.0,440900.0,454800.0,217800.0,305600.0,170200.0
3,2011-04-01,4.9,0.10,224.906,469500.0,177700.0,109600.0,152000.0,79300.0,117600.0,...,211500.0,225700.0,229800.0,161600.0,185900.0,457500.0,463300.0,191200.0,342300.0,184800.0
4,2011-05-01,4.9,0.09,225.964,471300.0,167900.0,108200.0,162900.0,78600.0,113800.0,...,215300.0,230800.0,260100.0,131500.0,175900.0,470600.0,475400.0,201500.0,342600.0,198900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2019-11-01,4.1,1.55,257.208,571500.0,264300.0,274400.0,316900.0,166600.0,174600.0,...,264100.0,370700.0,402700.0,343700.0,297200.0,557900.0,530300.0,279400.0,396600.0,250200.0
107,2019-12-01,4.1,1.55,256.974,575100.0,270000.0,277500.0,307100.0,172700.0,176600.0,...,265200.0,412500.0,454400.0,321500.0,292100.0,577100.0,579600.0,287100.0,398200.0,270300.0
108,2020-01-01,4.1,1.55,257.971,571700.0,281400.0,278900.0,321500.0,175300.0,170800.0,...,272300.0,408900.0,499200.0,273700.0,275100.0,557900.0,539400.0,305300.0,404100.0,260200.0
109,2020-02-01,4.0,1.58,258.678,568300.0,302900.0,283000.0,315500.0,173600.0,173100.0,...,248800.0,409000.0,513400.0,314400.0,265400.0,565600.0,609700.0,325600.0,397100.0,224300.0


### LOAD/CLEAN UNEMPLOYMENT RATE DATA

In [209]:
unemployment_file = Path("./resources/unemployment_data/unemployment_data_usa_only.csv")
unemployment_df = pd.read_csv(unemployment_file)
unemployment_df['TIME'] = pd.to_datetime(unemployment_df['TIME'], infer_datetime_format=True)
unemployment_df = unemployment_df.rename(columns={'TIME': 'DATE', 'Value': 'UNEMPLOYMENT_RATE'})
print(unemployment_df.dtypes)
unemployment_df

DATE                 datetime64[ns]
UNEMPLOYMENT_RATE           float64
dtype: object


,DATE,UNEMPLOYMENT_RATE
0,2010-11-01,9.8
1,2010-12-01,9.3
2,2011-01-01,9.1
3,2011-02-01,9.0
4,2011-03-01,9.0
...,...,...
131,2021-10-01,4.6
132,2021-11-01,4.2
133,2021-12-01,3.9
134,2022-01-01,4.0


In [210]:
# merge unemployment rate data into transposed dataset
time_transposed = time_transposed.merge(unemployment_df, how='inner', on='DATE')
unemp = time_transposed['UNEMPLOYMENT_RATE']
time_transposed = time_transposed.drop(columns=['UNEMPLOYMENT_RATE'])
time_transposed.insert(loc=1, column='UNEMPLOYMENT_RATE', value=unemp)
time_transposed

,DATE,UNEMPLOYMENT_RATE,AVG_MORTGAGE_RATE,INTEREST_RATES,INFLATION,10001,60601,89101,75201,43085,...,19468,32550,96738,76262,85641,34228,18938,95223,19047,60081
0,2011-01-01,9.1,4.8,0.17,220.223,462700.0,188500.0,115600.0,191900.0,89100.0,...,229700.0,284400.0,237800.0,128400.0,204100.0,432300.0,528100.0,266300.0,272300.0,203600.0
1,2011-02-01,9.0,4.9,0.16,221.309,466200.0,189400.0,114000.0,184500.0,86000.0,...,216700.0,258100.0,216000.0,148400.0,194000.0,476900.0,482600.0,244200.0,291500.0,175600.0
2,2011-03-01,9.0,5.0,0.14,223.467,465600.0,186300.0,110800.0,174500.0,79800.0,...,199800.0,253800.0,208800.0,150500.0,192400.0,440900.0,454800.0,217800.0,305600.0,170200.0
3,2011-04-01,9.1,4.9,0.10,224.906,469500.0,177700.0,109600.0,152000.0,79300.0,...,211500.0,225700.0,229800.0,161600.0,185900.0,457500.0,463300.0,191200.0,342300.0,184800.0
4,2011-05-01,9.0,4.9,0.09,225.964,471300.0,167900.0,108200.0,162900.0,78600.0,...,215300.0,230800.0,260100.0,131500.0,175900.0,470600.0,475400.0,201500.0,342600.0,198900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2019-11-01,3.6,4.1,1.55,257.208,571500.0,264300.0,274400.0,316900.0,166600.0,...,264100.0,370700.0,402700.0,343700.0,297200.0,557900.0,530300.0,279400.0,396600.0,250200.0
107,2019-12-01,3.6,4.1,1.55,256.974,575100.0,270000.0,277500.0,307100.0,172700.0,...,265200.0,412500.0,454400.0,321500.0,292100.0,577100.0,579600.0,287100.0,398200.0,270300.0
108,2020-01-01,3.5,4.1,1.55,257.971,571700.0,281400.0,278900.0,321500.0,175300.0,...,272300.0,408900.0,499200.0,273700.0,275100.0,557900.0,539400.0,305300.0,404100.0,260200.0
109,2020-02-01,3.5,4.0,1.58,258.678,568300.0,302900.0,283000.0,315500.0,173600.0,...,248800.0,409000.0,513400.0,314400.0,265400.0,565600.0,609700.0,325600.0,397100.0,224300.0


### LOAD/CLEAN GAS PRICE DATA

In [211]:
gas_file = Path('./resources/gas_price/gas_price.csv')
gas_df = pd.read_csv(gas_file)
gas_df['DATE'] = pd.to_datetime(gas_df['DATE'], infer_datetime_format=True)
print(gas_df.dtypes)
gas_df

DATE                datetime64[ns]
PRICE_PER_GALLON           float64
dtype: object


,DATE,PRICE_PER_GALLON
0,1993-04-01,1.078
1,1993-05-01,1.100
2,1993-06-01,1.097
3,1993-07-01,1.078
4,1993-08-01,1.062
...,...,...
342,2021-10-01,3.384
343,2021-11-01,3.491
344,2021-12-01,3.406
345,2022-01-01,3.413


In [212]:
# add gas price data to our master datasource
time_transposed = time_transposed.merge(gas_df, how='inner', on='DATE')
gas = time_transposed['PRICE_PER_GALLON']
time_transposed = time_transposed.drop(columns=['PRICE_PER_GALLON'])
time_transposed.insert(loc=1, column='PRICE_PER_GALLON', value=gas)
time_transposed = time_transposed.dropna()
time_transposed

,DATE,PRICE_PER_GALLON,UNEMPLOYMENT_RATE,AVG_MORTGAGE_RATE,INTEREST_RATES,INFLATION,10001,60601,89101,75201,...,19468,32550,96738,76262,85641,34228,18938,95223,19047,60081
0,2011-01-01,3.148,9.1,4.8,0.17,220.223,462700.0,188500.0,115600.0,191900.0,...,229700.0,284400.0,237800.0,128400.0,204100.0,432300.0,528100.0,266300.0,272300.0,203600.0
1,2011-02-01,3.264,9.0,4.9,0.16,221.309,466200.0,189400.0,114000.0,184500.0,...,216700.0,258100.0,216000.0,148400.0,194000.0,476900.0,482600.0,244200.0,291500.0,175600.0
2,2011-03-01,3.615,9.0,5.0,0.14,223.467,465600.0,186300.0,110800.0,174500.0,...,199800.0,253800.0,208800.0,150500.0,192400.0,440900.0,454800.0,217800.0,305600.0,170200.0
3,2011-04-01,3.852,9.1,4.9,0.10,224.906,469500.0,177700.0,109600.0,152000.0,...,211500.0,225700.0,229800.0,161600.0,185900.0,457500.0,463300.0,191200.0,342300.0,184800.0
4,2011-05-01,3.960,9.0,4.9,0.09,225.964,471300.0,167900.0,108200.0,162900.0,...,215300.0,230800.0,260100.0,131500.0,175900.0,470600.0,475400.0,201500.0,342600.0,198900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2019-11-01,2.693,3.6,4.1,1.55,257.208,571500.0,264300.0,274400.0,316900.0,...,264100.0,370700.0,402700.0,343700.0,297200.0,557900.0,530300.0,279400.0,396600.0,250200.0
107,2019-12-01,2.645,3.6,4.1,1.55,256.974,575100.0,270000.0,277500.0,307100.0,...,265200.0,412500.0,454400.0,321500.0,292100.0,577100.0,579600.0,287100.0,398200.0,270300.0
108,2020-01-01,2.636,3.5,4.1,1.55,257.971,571700.0,281400.0,278900.0,321500.0,...,272300.0,408900.0,499200.0,273700.0,275100.0,557900.0,539400.0,305300.0,404100.0,260200.0
109,2020-02-01,2.533,3.5,4.0,1.58,258.678,568300.0,302900.0,283000.0,315500.0,...,248800.0,409000.0,513400.0,314400.0,265400.0,565600.0,609700.0,325600.0,397100.0,224300.0


In [213]:
# Create CSV with fully cleaned and incorporated data
time_transposed.to_csv('./resources/full_data_set_zipcodes.csv', index=False)

In [214]:
# Re-transpose the table for use with tableau
time_transposed = time_transposed.astype({'DATE': 'str'})
time_df = time_transposed.transpose()
time_df.reset_index(drop = False, inplace = True)
time_df = time_df.rename(columns=time_df.iloc[0])
# time_df = time_df.drop([0, 0])
time_df = time_df.rename(columns={"DATE": 'Feature'})
time_df

,Feature,2011-01-01,2011-02-01,2011-03-01,2011-04-01,2011-05-01,2011-06-01,2011-07-01,2011-08-01,2011-09-01,...,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01
0,DATE,2011-01-01,2011-02-01,2011-03-01,2011-04-01,2011-05-01,2011-06-01,2011-07-01,2011-08-01,2011-09-01,...,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01
1,PRICE_PER_GALLON,3.148,3.264,3.615,3.852,3.96,3.735,3.705,3.696,3.667,...,2.804,2.823,2.707,2.681,2.724,2.693,2.645,2.636,2.533,2.329
2,UNEMPLOYMENT_RATE,9.1,9.0,9.0,9.1,9.0,9.1,9.0,9.0,9.0,...,3.6,3.7,3.7,3.5,3.6,3.6,3.6,3.5,3.5,4.4
3,AVG_MORTGAGE_RATE,4.8,4.9,5.0,4.9,4.9,4.7,4.6,4.6,4.3,...,4.3,4.2,4.1,4.0,4.0,4.1,4.1,4.1,4.0,3.7
4,INTEREST_RATES,0.17,0.16,0.14,0.1,0.09,0.09,0.07,0.1,0.08,...,2.38,2.4,2.13,2.04,1.83,1.55,1.55,1.55,1.58,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,34228,432300.0,476900.0,440900.0,457500.0,470600.0,489500.0,464600.0,420600.0,414700.0,...,567400.0,554700.0,502000.0,495600.0,535300.0,557900.0,577100.0,557900.0,565600.0,579700.0
949,18938,528100.0,482600.0,454800.0,463300.0,475400.0,478600.0,491700.0,531700.0,509200.0,...,545100.0,548300.0,579100.0,556500.0,592700.0,530300.0,579600.0,539400.0,609700.0,608700.0
950,95223,266300.0,244200.0,217800.0,191200.0,201500.0,208400.0,208600.0,200800.0,211900.0,...,326500.0,317800.0,293100.0,281000.0,279100.0,279400.0,287100.0,305300.0,325600.0,345100.0
951,19047,272300.0,291500.0,305600.0,342300.0,342600.0,329400.0,297500.0,279400.0,294300.0,...,339800.0,371300.0,358600.0,366800.0,362400.0,396600.0,398200.0,404100.0,397100.0,416200.0


In [215]:
time_df.to_csv('./resources/time_df.csv', index=False)

## Make super skinny table for use with Tableau

In [216]:
# skinny_df = pd.DataFrame(columns=['DATE', 'Zip Code', 'PRICE_PER_GALLON', 'UNEMPLOYMENT_RATE', 'AVG_MORTGAGE_RATE', 'INTEREST_RATES', 'AVG_HOUSE_PRICE'])
# for index, row in time_transposed.iterrows():
#     skinny_df['DATE'].iloc[i] =
skinny_df = pd.melt(time_transposed, id_vars=['DATE', 'PRICE_PER_GALLON', 'UNEMPLOYMENT_RATE', 'AVG_MORTGAGE_RATE', 'INTEREST_RATES', 'INFLATION'], value_vars=[i for i in time_transposed.columns if i != 'DATE' \
                                                                and i != 'PRICE_PER_GALLON' and i != 'UNEMPLOYMENT_RATE' and i != 'AVG_MORTGAGE_RATE' \
                                                                and i != 'INTEREST_RATES' and i != 'INFLATION'])
skinny_df = skinny_df.rename(columns={'variable': 'ZIP_CODE', 'value': 'AVG_HOUSE_PRICE'})
zipcode = skinny_df['ZIP_CODE']
skinny_df = skinny_df.drop(columns=['ZIP_CODE'])
skinny_df.insert(loc=0, column='ZIP_CODE', value=zipcode)
skinny_df

,ZIP_CODE,DATE,PRICE_PER_GALLON,UNEMPLOYMENT_RATE,AVG_MORTGAGE_RATE,INTEREST_RATES,INFLATION,AVG_HOUSE_PRICE
0,10001,2011-01-01,3.148,9.1,4.8,0.17,220.223,462700.0
1,10001,2011-02-01,3.264,9.0,4.9,0.16,221.309,466200.0
2,10001,2011-03-01,3.615,9.0,5.0,0.14,223.467,465600.0
3,10001,2011-04-01,3.852,9.1,4.9,0.10,224.906,469500.0
4,10001,2011-05-01,3.960,9.0,4.9,0.09,225.964,471300.0
...,...,...,...,...,...,...,...,...
105112,60081,2019-11-01,2.693,3.6,4.1,1.55,257.208,250200.0
105113,60081,2019-12-01,2.645,3.6,4.1,1.55,256.974,270300.0
105114,60081,2020-01-01,2.636,3.5,4.1,1.55,257.971,260200.0
105115,60081,2020-02-01,2.533,3.5,4.0,1.58,258.678,224300.0


In [217]:
# export tableau ready table to csv
skinny_df.to_csv('./resources/full_data_set_long.csv', index=False)